In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain_community.cache import RedisSemanticCache
import json
from langchain_core.outputs import Generation
REDIS_URI: str = "localhost:6378"

embeddings = OpenAIEmbeddings(model="text-embedding-nomic-embed-text-v1.5", openai_api_base="http://localhost:4000", openai_api_key="sk-llmops")
llm_chat = init_chat_model(
            "qwen-instruct",
            api_key="sk-llmops",
            temperature=0.7,
            model_provider="openai",
            base_url="http://localhost:4000",
    )

In [35]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
print(f"VERTEX AI EMBEDDINGS")
print(query_result[:5])

VERTEX AI EMBEDDINGS
[0.018747683614492416, 0.03083343431353569, -0.15383435785770416, -0.03006676211953163, 0.012376675382256508]


In [36]:
semantic_cache = RedisSemanticCache(
            redis_url="redis://localhost:6378",
            embedding=embeddings
        )

In [40]:
question = "Hello, what your name?"

# Check if the question is already cached
cached_response = semantic_cache.lookup(question, 'pre-cache')
if cached_response:
    print(cached_response[0].text)
else:
    response = llm_chat.invoke(question)
    response_dict = response.model_dump()
    print(json.dumps(response_dict, indent=2, ensure_ascii=False))

    semantic_cache.update(question, 'pre-cache', [
    Generation(text=str(response.content))])


I'm Qwen, an AI assistant created by Alibaba Cloud. How can I help you today?
